In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from speckit.systems import (
    SISO_optimal_spectral_analysis,
    MISO_analytic_optimal_spectral_analysis,
    MISO_numeric_optimal_spectral_analysis,
)
from speckit import compute_spectrum
from speckit.plotting import default_rc

plt.rcParams.update(default_rc)


# Systems: Optimal Spectral Analysis

This notebook demonstrates the three optimal spectral analysis functions in `systems.py`:

1. **SISO_optimal_spectral_analysis**: Single-Input Single-Output system analysis
2. **MISO_analytic_optimal_spectral_analysis**: Multiple-Input Single-Output system analysis using analytic solution
3. **MISO_numeric_optimal_spectral_analysis**: Multiple-Input Single-Output system analysis using numeric solution

These functions estimate the amplitude spectral density (ASD) of the output signal with the influence of input signals subtracted, providing optimal spectral analysis for system identification.


In [ ]:
# Helper function to create synthetic SISO system data
def create_siso_data(n=100000, fs=1.0, noise_level=0.1, seed=42):
    """Create synthetic Single-Input Single-Output system data.
    
    System: output = H * input + noise
    where H is a transfer function (frequency-dependent gain)
    """
    np.random.seed(seed)
    t = np.arange(n) / fs
    
    # Create input signal: combination of sinusoids
    f1, f2, f3 = 0.01, 0.05, 0.1  # Hz
    input_signal = (
        np.sin(2 * np.pi * f1 * t) +
        0.5 * np.sin(2 * np.pi * f2 * t) +
        0.3 * np.sin(2 * np.pi * f3 * t)
    )
    
    # Create output: input with some gain + noise
    # Simple gain of 2.5
    output_signal = 2.5 * input_signal + noise_level * np.random.randn(n)
    
    return input_signal, output_signal, fs


In [ ]:
# Helper function to create synthetic MISO system data
def create_miso_data(n=100000, fs=1.0, noise_level=0.1, n_inputs=3, seed=42):
    """Create synthetic Multiple-Input Single-Output system data.
    
    System: output = H1*input1 + H2*input2 + ... + noise
    """
    np.random.seed(seed)
    t = np.arange(n) / fs
    
    inputs = []
    gains = [1.5, -0.8, 2.0, 0.5, -1.2][:n_inputs]  # Different gains for each input
    
    for i in range(n_inputs):
        # Each input has different frequency content
        f = 0.01 * (i + 1)
        input_signal = np.sin(2 * np.pi * f * t) + 0.3 * np.sin(2 * np.pi * f * 2 * t)
        inputs.append(input_signal)
    
    # Create output: weighted sum of inputs + noise
    output_signal = np.zeros(n)
    for i, (inp, gain) in enumerate(zip(inputs, gains)):
        output_signal += gain * inp
    output_signal += noise_level * np.random.randn(n)
    
    return inputs, output_signal, fs


## 1. SISO Optimal Spectral Analysis

`SISO_optimal_spectral_analysis` performs optimal spectral analysis on a Single-Input Single-Output system. It estimates the ASD of the output with the influence of the input subtracted.


In [ ]:
# Generate SISO data
input_siso, output_siso, fs_siso = create_siso_data(n=200000, fs=1.0, noise_level=0.15)

# Plot time series
fig, axes = plt.subplots(2, 1, figsize=(10, 6), sharex=True)
t_plot = np.arange(len(input_siso[:5000])) / fs_siso
axes[0].plot(t_plot, input_siso[:5000], label="Input", alpha=0.7)
axes[0].set_ylabel("Amplitude")
axes[0].set_title("SISO System: Input Signal")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(t_plot, output_siso[:5000], label="Output", color="C1", alpha=0.7)
axes[1].set_xlabel("Time (s)")
axes[1].set_ylabel("Amplitude")
axes[1].set_title("SISO System: Output Signal")
axes[1].legend()
axes[1].grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# Compute optimal spectral analysis
print("Computing SISO optimal spectral analysis...")
f_siso, asd_optimal_siso = SISO_optimal_spectral_analysis(
    input_siso, output_siso, fs_siso, Lmin=10000, order=-1, win="hann", olap=0.5
)

# Also compute regular spectrum of output for comparison
result_output = compute_spectrum(output_siso, fs_siso, Lmin=10000, order=-1, win="hann", olap=0.5)
asd_output = np.sqrt(result_output.Gxx)

# Plot comparison
fig, ax = plt.subplots(figsize=(8, 5))
ax.loglog(f_siso, asd_output, label="Output ASD (with input influence)", 
          color="C0", alpha=0.7, linestyle="--")
ax.loglog(f_siso, asd_optimal_siso, label="Optimal ASD (input influence removed)", 
          color="C1", linewidth=2)
ax.set_xlabel("Frequency (Hz)")
ax.set_ylabel("ASD (units/√Hz)")
ax.set_title("SISO Optimal Spectral Analysis")
ax.legend()
ax.grid(True, alpha=0.3, which="both")
plt.tight_layout()
plt.show()

print(f"Frequency range: {f_siso[0]:.6f} - {f_siso[-1]:.3f} Hz")
print(f"Number of frequency points: {len(f_siso)}")


## 2. MISO Analytic Optimal Spectral Analysis

`MISO_analytic_optimal_spectral_analysis` performs optimal spectral analysis on a Multiple-Input Single-Output system using an exact analytic solution. This is suitable for systems with up to ~5 inputs.


In [ ]:
# Generate MISO data with 3 inputs
inputs_miso, output_miso, fs_miso = create_miso_data(
    n=200000, fs=1.0, noise_level=0.2, n_inputs=3
)

# Plot time series
fig, axes = plt.subplots(4, 1, figsize=(10, 8), sharex=True)
t_plot = np.arange(len(inputs_miso[0][:5000])) / fs_miso

for i, inp in enumerate(inputs_miso):
    axes[i].plot(t_plot, inp[:5000], label=f"Input {i+1}", alpha=0.7)
    axes[i].set_ylabel("Amplitude")
    axes[i].set_title(f"MISO System: Input {i+1}")
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

axes[3].plot(t_plot, output_miso[:5000], label="Output", color="C3", alpha=0.7)
axes[3].set_xlabel("Time (s)")
axes[3].set_ylabel("Amplitude")
axes[3].set_title("MISO System: Output Signal")
axes[3].legend()
axes[3].grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# Compute MISO analytic optimal spectral analysis
print("Computing MISO analytic optimal spectral analysis...")
f_miso_analytic, asd_optimal_miso_analytic = MISO_analytic_optimal_spectral_analysis(
    inputs_miso, output_miso, fs_miso, order=-1, win="hann", olap=0.5
)

# Also compute regular spectrum of output for comparison
result_output_miso = compute_spectrum(
    output_miso, fs_miso, order=-1, win="hann", olap=0.5
)
asd_output_miso = np.sqrt(result_output_miso.Gxx)

# Plot comparison
fig, ax = plt.subplots(figsize=(8, 5))
ax.loglog(f_miso_analytic, asd_output_miso, 
          label="Output ASD (with inputs influence)", 
          color="C0", alpha=0.7, linestyle="--")
ax.loglog(f_miso_analytic, asd_optimal_miso_analytic, 
          label="Optimal ASD (inputs influence removed, analytic)", 
          color="C1", linewidth=2)
ax.set_xlabel("Frequency (Hz)")
ax.set_ylabel("ASD (units/√Hz)")
ax.set_title("MISO Analytic Optimal Spectral Analysis (3 inputs)")
ax.legend()
ax.grid(True, alpha=0.3, which="both")
plt.tight_layout()
plt.show()

print(f"Frequency range: {f_miso_analytic[0]:.6f} - {f_miso_analytic[-1]:.3f} Hz")
print(f"Number of frequency points: {len(f_miso_analytic)}")


## 3. MISO Numeric Optimal Spectral Analysis

`MISO_numeric_optimal_spectral_analysis` performs optimal spectral analysis on a Multiple-Input Single-Output system using a numeric solution. This is more suitable for systems with many inputs (>5) or when the analytic solution becomes computationally expensive.


In [ ]:
# Compute MISO numeric optimal spectral analysis
print("Computing MISO numeric optimal spectral analysis...")
f_miso_numeric, asd_optimal_miso_numeric = MISO_numeric_optimal_spectral_analysis(
    inputs_miso, output_miso, fs_miso, order=-1, win="hann", olap=0.5
)

# Plot comparison
fig, ax = plt.subplots(figsize=(8, 5))
ax.loglog(f_miso_numeric, asd_output_miso, 
          label="Output ASD (with inputs influence)", 
          color="C0", alpha=0.7, linestyle="--")
ax.loglog(f_miso_numeric, asd_optimal_miso_numeric, 
          label="Optimal ASD (inputs influence removed, numeric)", 
          color="C2", linewidth=2)
ax.set_xlabel("Frequency (Hz)")
ax.set_ylabel("ASD (units/√Hz)")
ax.set_title("MISO Numeric Optimal Spectral Analysis (3 inputs)")
ax.legend()
ax.grid(True, alpha=0.3, which="both")
plt.tight_layout()
plt.show()

print(f"Frequency range: {f_miso_numeric[0]:.6f} - {f_miso_numeric[-1]:.3f} Hz")
print(f"Number of frequency points: {len(f_miso_numeric)}")


## Comparison: Analytic vs Numeric MISO Solutions

Let's compare the analytic and numeric solutions for the same MISO system to verify they produce similar results.


In [ ]:
# Compare analytic and numeric solutions
fig, axes = plt.subplots(2, 1, figsize=(10, 8), sharex=True)

# Plot both solutions
axes[0].loglog(f_miso_analytic, asd_optimal_miso_analytic, 
               label="Analytic solution", color="C1", linewidth=2)
axes[0].loglog(f_miso_numeric, asd_optimal_miso_numeric, 
               label="Numeric solution", color="C2", linewidth=2, linestyle="--")
axes[0].set_ylabel("ASD (units/√Hz)")
axes[0].set_title("MISO Optimal ASD: Analytic vs Numeric")
axes[0].legend()
axes[0].grid(True, alpha=0.3, which="both")

# Plot difference (relative)
# Interpolate to common frequency grid for comparison
f_common = f_miso_analytic
asd_analytic_interp = np.interp(f_common, f_miso_analytic, asd_optimal_miso_analytic)
asd_numeric_interp = np.interp(f_common, f_miso_numeric, asd_optimal_miso_numeric)

# Avoid division by zero
mask = asd_analytic_interp > 0
relative_diff = np.zeros_like(f_common)
relative_diff[mask] = np.abs(asd_analytic_interp[mask] - asd_numeric_interp[mask]) / asd_analytic_interp[mask]

axes[1].semilogx(f_common[mask], relative_diff[mask] * 100, 
                 color="C3", linewidth=1.5)
axes[1].set_xlabel("Frequency (Hz)")
axes[1].set_ylabel("Relative Difference (%)")
axes[1].set_title("Relative Difference: |Analytic - Numeric| / Analytic")
axes[1].grid(True, alpha=0.3, which="both")
plt.tight_layout()
plt.show()

print(f"Mean relative difference: {np.mean(relative_diff[mask]) * 100:.4f}%")
print(f"Max relative difference: {np.max(relative_diff[mask]) * 100:.4f}%")


## Example: MISO with More Inputs

The numeric solution is particularly useful for systems with many inputs. Let's demonstrate with 5 inputs.


In [ ]:
# Generate MISO data with 5 inputs
inputs_miso_5, output_miso_5, fs_miso_5 = create_miso_data(
    n=200000, fs=1.0, noise_level=0.2, n_inputs=5
)

# Compute using numeric method (analytic would be slow for 5 inputs)
print("Computing MISO numeric optimal spectral analysis (5 inputs)...")
f_miso_5, asd_optimal_miso_5 = MISO_numeric_optimal_spectral_analysis(
    inputs_miso_5, output_miso_5, fs_miso_5, order=-1, win="hann", olap=0.5
)

# Also compute regular spectrum of output for comparison
result_output_miso_5 = compute_spectrum(
    output_miso_5, fs_miso_5, order=-1, win="hann", olap=0.5
)
asd_output_miso_5 = np.sqrt(result_output_miso_5.Gxx)

# Plot comparison
fig, ax = plt.subplots(figsize=(8, 5))
ax.loglog(f_miso_5, asd_output_miso_5, 
          label="Output ASD (with inputs influence)", 
          color="C0", alpha=0.7, linestyle="--")
ax.loglog(f_miso_5, asd_optimal_miso_5, 
          label="Optimal ASD (inputs influence removed, numeric)", 
          color="C2", linewidth=2)
ax.set_xlabel("Frequency (Hz)")
ax.set_ylabel("ASD (units/√Hz)")
ax.set_title("MISO Numeric Optimal Spectral Analysis (5 inputs)")
ax.legend()
ax.grid(True, alpha=0.3, which="both")
plt.tight_layout()
plt.show()

print(f"Frequency range: {f_miso_5[0]:.6f} - {f_miso_5[-1]:.3f} Hz")
print(f"Number of frequency points: {len(f_miso_5)}")


## Summary

This notebook demonstrated the three optimal spectral analysis functions:

1. **SISO_optimal_spectral_analysis**: For single-input single-output systems, removes the influence of one input signal from the output spectrum.

2. **MISO_analytic_optimal_spectral_analysis**: For multiple-input single-output systems, uses symbolic computation to solve the system of equations analytically. Best for systems with ≤5 inputs.

3. **MISO_numeric_optimal_spectral_analysis**: For multiple-input single-output systems, solves the system of equations numerically. More efficient for systems with many inputs (>5).

All three functions estimate the residual ASD of the output after subtracting the influence of the input signals, which is useful for:
- System identification
- Noise characterization
- Understanding residual signals after known inputs are accounted for
